In [1]:
import pandas as pd
import numpy as np
from data_work import processed
df=pd.read_csv("/Users/mac/Dev/data/dta_IoT/new2test.csv")
#delete all null columns
id=[16,17,21,22,23,24]
col=df.columns
for idx in id:
    df=df.drop(col[idx],axis=1)
data=processed(df,"subcategory ")

In [2]:
data

,subcategory,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,...,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,category
0,6,5425,2934,6,2,0,184,20,901,8,...,1253,5,3,86,12,108,31,15,1,0
1,2,3463,6036,0,2,2,2366,24,609,9,...,582,5,4,507,32,2124,1589,1334,1,3
2,3,1890,1801,0,3,11,996,26,697,1,...,0,1,0,41,0,112,276,0,0,1
3,7,6078,3586,0,3,0,356,25,901,9,...,2029,9,0,71,0,368,552,0,1,0
4,3,1244,1162,0,2,5,4755,3,970,57,...,2545,42,42,535,173,1315,1214,1003,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7021,3,1304,1215,0,3,2,770,27,697,1,...,0,1,0,41,0,555,740,0,0,1
7022,2,3436,6010,0,2,2,1493,24,210,9,...,298,5,4,64,54,2421,1685,1420,1,3
7023,1,4747,5103,0,2,2,3979,20,900,9,...,258,5,4,253,81,632,446,504,1,0
7024,7,6090,3598,0,3,0,370,25,901,9,...,2029,9,0,71,0,368,552,0,1,0


In [3]:
import torch.nn.functional as F
import torch
labels=torch.tensor(data["subcategory "])
labels_one_hot = F.one_hot(labels, 8)
labels_one_hot 



tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0]])

In [4]:
label_counts = torch.sum(labels_one_hot, dim=0)
num_labels = label_counts.size(0)
for label_idx in range(num_labels):
    label_count = label_counts[label_idx].item()
    print(f"Label {label_idx}: {label_count} samples")

Label 0: 114 samples
Label 1: 994 samples
Label 2: 995 samples
Label 3: 931 samples
Label 4: 992 samples
Label 5: 1000 samples
Label 6: 1000 samples
Label 7: 1000 samples


In [5]:
X=data.drop("subcategory ",axis=1)
X=torch.tensor(X.values)
data=torch.cat((X, labels_one_hot), dim=1)

In [27]:
import utils, torch, time, os, pickle
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, input_dim=100, output_dim=1, input_size=32, class_num=8):
        super(generator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128 ,self.output_dim),
            nn.Tanh()
        )
        utils.initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        return x

class discriminator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    def __init__(self, input_dim=1, output_dim=1, input_size=32, class_num=8):
        super(discriminator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128 ,self.output_dim),
            nn.Sigmoid(),
        )
        utils.initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        return x


class CGAN(object):
    def __init__(self, args):
        # parameters
        self.epoch = args.epoch
        self.batch_size = args.batch_size
        self.save_dir = args.save_dir
        self.result_dir = args.result_dir
        self.dataset = args.dataset
        self.log_dir = args.log_dir
        self.gpu_mode = args.gpu_mode
        self.model_name = args.gan_type
        self.input_size = args.input_size
        self.z_dim = args.z_dim
        self.class_num = args.n_class
        self.sample_num = self.class_num ** 2 
        # load dataset
        self.data_loader = DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)
        data=next(iter(self.data_loader))
        #option
        dim=int(data.shape[1])-8

        # networks init
        self.G = generator(input_dim=self.z_dim, output_dim=dim, input_size=self.input_size, class_num=self.class_num)
        self.D = discriminator(input_dim=dim, output_dim=1, input_size=self.input_size, class_num=self.class_num)
        self.G_optimizer = optim.Adam(self.G.parameters(), lr=args.lrG)
        self.D_optimizer = optim.Adam(self.D.parameters(), lr=args.lrD)
        
        self.BCE_loss = nn.BCELoss()

        print('---------- Networks architecture -------------')
        utils.print_network(self.G)
        utils.print_network(self.D)
        print('-----------------------------------------------')


        # fixed noise & condition
        self.sample_z_ = torch.zeros((self.sample_num, self.z_dim)) # z_dim tượng trưng cho noise vector, vector nhiều có kích thuóc dài bằng vector onehotencoded^2 của label 
        for i in range(self.class_num):
            self.sample_z_[i*self.class_num] = torch.rand(1, self.z_dim)#tạo một vector random có kích thuớc 1, z_dim ( chiều dài của vector nhiễu )
            for j in range(1, self.class_num):
                self.sample_z_[i*self.class_num + j] = self.sample_z_[i*self.class_num]

        temp = torch.zeros((self.class_num, 1))
        for i in range(self.class_num):
            temp[i, 0] = i

        temp_y = torch.zeros((self.sample_num, 1))
        for i in range(self.class_num):
            temp_y[i*self.class_num: (i+1)*self.class_num] = temp

        self.sample_y_ = torch.zeros((self.sample_num, self.class_num)).scatter_(1, temp_y.type(torch.LongTensor), 1)


    def train(self):
        self.train_hist = {}
        self.train_hist['D_loss'] = []
        self.train_hist['G_loss'] = []
        self.train_hist['per_epoch_time'] = []
        self.train_hist['total_time'] = []

        self.y_real_, self.y_fake_ = torch.ones(self.batch_size, 1), torch.zeros(self.batch_size, 1)

        self.D.train()
        print('training start!!')
        start_time = time.time()
        for epoch in range(self.epoch):
            self.G.train()
            epoch_start_time = time.time()
            for iter, da in enumerate(self.data_loader):
                x_=da[:,:28].float()
                y_=da[:,-8:].float()
                if iter == self.data_loader.dataset.__len__() // self.batch_size:
                    break

                z_ = torch.rand((self.batch_size, self.z_dim))
                # update D network
                self.D.train()
                self.G.eval()
                self.D_optimizer.zero_grad()

                D_real = self.D(x_, y_)
                D_real_loss = self.BCE_loss(D_real, self.y_real_)
                G_ = self.G(z_, y_)
                D_fake = self.D(G_, y_)
                D_fake_loss = self.BCE_loss(D_fake, self.y_fake_)

                D_loss = D_real_loss + D_fake_loss
                self.train_hist['D_loss'].append(D_loss.item())

                D_loss.backward()
                self.D_optimizer.step()

                 # update G network
                self.D.eval()
                self.G.train()
                self.G_optimizer.zero_grad()

                z_=torch.rand((self.batch_size, self.z_dim))
                y = torch.randint(low=0, high=8, size=(self.batch_size,))
                y_label_ = F.one_hot(y, 8)
                G_ = self.G(z_, y_label_)
                D_fake = self.D(G_, y_label_)
                G_loss = self.BCE_loss(D_fake, self.y_real_)
                self.train_hist['G_loss'].append(G_loss.item())

                G_loss.backward()
                self.G_optimizer.step()

               
                if ((iter + 1) % 100) == 0:
                    print("Epoch: [%2d] [%4d/%4d] D_loss: %.8f, G_loss: %.8f" %
                          ((epoch + 1), (iter + 1), self.data_loader.dataset.__len__() // self.batch_size, D_loss.item(), G_loss.item()))

            self.train_hist['per_epoch_time'].append(time.time() - epoch_start_time)
                

        self.train_hist['total_time'].append(time.time() - start_time)
        print("Avg one epoch time: %.2f, total %d epochs time: %.2f" % (np.mean(self.train_hist['per_epoch_time']),
              self.epoch, self.train_hist['total_time'][0]))
        print("Training finish!")


In [32]:
class Args:
    def __init__(self,data):
        self.epoch = 10
        self.batch_size = 32
        self.save_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/model.h5'
        self.result_dir = '/Users/mac/Dev/source/article/FerderatedLearning/federated_learning/NonIID/result.txt'
        self.dataset = data
        self.log_dir = 'logs/'
        self.gpu_mode = True
        self.gan_type = 'cGAN'
        self.input_size = 3200
        self.z_dim = 2
        self.n_class = 8
        self.sample_num = self.n_class ** 2
        self.lrG=0.0001
        self.lrD=0.0001

In [33]:
arg=Args(data)
print(arg.dataset)

tensor([[5425, 2934,    6,  ...,    0,    1,    0],
        [3463, 6036,    0,  ...,    0,    0,    0],
        [1890, 1801,    0,  ...,    0,    0,    0],
        ...,
        [4747, 5103,    0,  ...,    0,    0,    0],
        [6090, 3598,    0,  ...,    0,    0,    1],
        [5356, 2865,    6,  ...,    0,    1,    0]])


In [34]:
model=CGAN(arg)

---------- Networks architecture -------------
generator(
  (fc): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=28, bias=True)
    (7): Tanh()
  )
)
Total number of parameters: 12892
discriminator(
  (fc): Sequential(
    (0): Linear(in_features=36, out_features=64, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): ReLU()
    (5): Linear(in_features=128, out_features=1, bias=True)
    (6): Sigmoid()
  )
)
Total number of parameters: 10817
-----------------------------------------------


In [35]:
model.train()

training start!!
Epoch: [ 1] [ 100/ 219] D_loss: 0.68373287, G_loss: 0.70363426
Epoch: [ 1] [ 200/ 219] D_loss: 0.67075890, G_loss: 0.71750754
Epoch: [ 2] [ 100/ 219] D_loss: 0.63447922, G_loss: 0.75497872
Epoch: [ 2] [ 200/ 219] D_loss: 0.57962519, G_loss: 0.82375431
Epoch: [ 3] [ 100/ 219] D_loss: 0.50504225, G_loss: 0.92836130
Epoch: [ 3] [ 200/ 219] D_loss: 0.39176321, G_loss: 1.13723028
Epoch: [ 4] [ 100/ 219] D_loss: 0.27487636, G_loss: 1.39896297
Epoch: [ 4] [ 200/ 219] D_loss: 0.21980664, G_loss: 1.66701269
Epoch: [ 5] [ 100/ 219] D_loss: 0.13627894, G_loss: 2.08880162
Epoch: [ 5] [ 200/ 219] D_loss: 0.12185574, G_loss: 2.18447304
Epoch: [ 6] [ 100/ 219] D_loss: 0.07491970, G_loss: 2.62214351
Epoch: [ 6] [ 200/ 219] D_loss: 0.05226010, G_loss: 2.88647676
Epoch: [ 7] [ 100/ 219] D_loss: 0.04796908, G_loss: 3.04335070
Epoch: [ 7] [ 200/ 219] D_loss: 0.03701843, G_loss: 3.40809345
Epoch: [ 8] [ 100/ 219] D_loss: 0.02085172, G_loss: 3.81437182
Epoch: [ 8] [ 200/ 219] D_loss: 0.0186

In [36]:
y=torch.tensor([1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0,1,2,3,4,5,6,7,0])
y=F.one_hot(y, 8)



In [37]:
y

tensor([[0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0]])

In [38]:
len=y.shape[0]
len

24

In [39]:
z= torch.rand(len,2)
y=torch.tensor(y)

/var/folders/0j/2h4f0bj12r775fg_s2v4hpqh0000gn/T/ipykernel_40440/949614172.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y)


In [40]:
z

tensor([[0.1450, 0.2414],
        [0.6399, 0.0262],
        [0.4495, 0.3421],
        [0.6839, 0.8740],
        [0.8144, 0.7765],
        [0.4800, 0.0614],
        [0.8289, 0.0341],
        [0.2718, 0.7847],
        [0.4388, 0.6797],
        [0.1786, 0.4666],
        [0.7619, 0.8936],
        [0.4904, 0.3848],
        [0.5010, 0.0805],
        [0.8361, 0.0950],
        [0.2121, 0.3462],
        [0.3835, 0.8032],
        [0.8578, 0.8176],
        [0.5582, 0.2958],
        [0.0874, 0.1209],
        [0.6398, 0.9048],
        [0.2104, 0.7998],
        [0.5815, 0.7665],
        [0.5457, 0.7531],
        [0.0763, 0.6362]])

In [41]:
mm=torch.cat([z,y],1)

In [42]:
mm

tensor([[0.1450, 0.2414, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.6399, 0.0262, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4495, 0.3421, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.6839, 0.8740, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000],
        [0.8144, 0.7765, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000],
        [0.4800, 0.0614, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000],
        [0.8289, 0.0341, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.2718, 0.7847, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4388, 0.6797, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1786, 0.4666, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0

In [43]:
mm.shape[1]

10

In [44]:
data_fake=model.G(z,y)

In [45]:
data_fake

tensor([[ 0.9999, -0.9575,  0.6972,  0.7160, -0.7945,  0.9942,  0.7253,  0.9391,
         -0.6657,  0.1152,  0.4850,  1.0000,  0.9478,  0.9998,  0.9628,  0.8351,
          0.7791,  0.5601,  0.9918,  0.7914,  0.6807,  0.4362,  0.8381,  0.9571,
          0.8996,  0.7545,  0.7566, -0.8052],
        [ 0.9999, -0.9332,  0.8042,  0.6415, -0.7754,  0.9964,  0.6721,  0.8414,
         -0.5784,  0.6009,  0.1727,  0.9999,  0.9646,  0.9991,  0.9712,  0.8309,
          0.9155, -0.0598,  0.9675,  0.7141,  0.6152,  0.7843,  0.8892,  0.9812,
          0.9601,  0.8216,  0.7857, -0.8310],
        [ 0.9999, -0.9687,  0.6014,  0.6075, -0.7544,  0.9892,  0.7818,  0.9737,
         -0.5874, -0.1260,  0.6106,  1.0000,  0.9644,  0.9997,  0.9706,  0.7597,
          0.7117,  0.4759,  0.9913,  0.7541,  0.6196,  0.4679,  0.8173,  0.9412,
          0.8636,  0.7715,  0.6307, -0.7390],
        [ 1.0000, -0.9832,  0.6649,  0.6699, -0.8105,  0.9951,  0.8373,  0.9865,
         -0.6510, -0.1503,  0.6738,  1.0000,  0.9805